# Vote Average Prediction

### Predict movie average rating (vote_average_prediction) using linear regression.

### 1 Import libraries

In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


### Reading a cleaned dataset

In [12]:
df = pd.read_csv('../data/cleaned/TMDB_movie_dataset_cleaned.csv')

df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7060 entries, 0 to 7059
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    7060 non-null   int64  
 1   title                 7060 non-null   object 
 2   vote_average          7060 non-null   float64
 3   vote_count            7060 non-null   int64  
 4   status                7060 non-null   object 
 5   release_date          7060 non-null   object 
 6   revenue               7060 non-null   float64
 7   runtime               7060 non-null   int64  
 8   adult                 7060 non-null   bool   
 9   budget                7060 non-null   float64
 10  original_language     7060 non-null   object 
 11  popularity            7060 non-null   float64
 12  genres                7060 non-null   object 
 13  production_companies  7060 non-null   object 
 14  production_countries  7060 non-null   object 
 15  spoken_languages     

,id,vote_average,vote_count,revenue,runtime,budget,popularity,companies_count,release_year,release_month,release_day_of_week
count,7.060000e+03,7060.000000,7060.000000,7.060000e+03,7060.000000,7.060000e+03,7060.000000,7060.000000,7060.000000,7060.000000,7060.000000
mean,1.212015e+05,6.540372,2153.809490,9.172054e+07,109.731020,3.170544e+07,25.393574,3.543059,2001.889235,7.012323,3.156232
std,1.916880e+05,0.793936,3352.688912,1.789358e+08,20.685463,4.189299e+07,70.942102,2.386031,16.482109,3.384672,1.240843
min,5.000000e+00,1.908000,101.000000,1.000000e+00,5.000000,5.000000e+00,0.600000,0.000000,1915.000000,1.000000,0.000000
25%,9.387750e+03,6.031000,335.000000,8.339707e+06,95.000000,6.000000e+06,12.334000,2.000000,1995.000000,4.000000,2.000000
50%,1.628450e+04,6.566000,917.000000,3.011169e+07,106.000000,1.700000e+07,16.646000,3.000000,2006.000000,7.000000,3.000000
75%,1.643412e+05,7.107000,2387.500000,9.458670e+07,120.000000,4.000000e+07,24.814500,5.000000,2014.000000,10.000000,4.000000
max,1.040148e+06,8.707000,34495.000000,2.923706e+09,339.000000,4.600000e+08,2994.357000,30.000000,2023.000000,12.000000,6.000000


### Feature and target variable selection

In [13]:
numerical_features = ['budget', 'revenue', 'runtime', 'popularity', 'vote_count', 'companies_count',
                      'release_year', 'release_month', 'release_day_of_week']

categorical_features = ['original_language', 'adult']

X = df[numerical_features + categorical_features]
y = df['vote_average']


### Processing passes

In [15]:
X.loc[:, numerical_features] = X.loc[:, numerical_features].fillna(X[numerical_features].median())

X.loc[:, categorical_features] = X.loc[:, categorical_features].fillna(X[categorical_features].mode().iloc[0])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Creation of Transformers

In [16]:
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])


### Building a pipeline with a model

In [17]:
model = RandomForestRegressor(n_estimators=100, random_state=42)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', model)])

### Model training

In [18]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['budget', 'revenue',
                                                   'runtime', 'popularity',
                                                   'vote_count',
                                                   'companies_count',
                                                   'release_year',
                                                   'release_month',
                                                   'release_day_of_week']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['original_language',
                                                   'adult'])])),
                ('regressor', RandomForestRegressor(random_state=42))])

### Prediction

In [19]:
y_pred = pipeline.predict(X_test)

### Model evaluation

In [20]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

In [21]:
print(f"RMSE: {rmse:.3f}")
print(f"R2 Score: {r2:.3f}")

RMSE: 0.540
R2 Score: 0.520
